In [2]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point
import json
from shapely import to_geojson
from shapely.geometry import shape
from networkx.readwrite import json_graph
import folium

In [3]:
def graph_to_geojson(G, path):
    """
    Serialize NetworkX graph G (with Shapely geometries in node attrs)
    to a JSON file at 'path', converting geometries via to_geojson.
    """
    # 1. Extract node-link data
    data = json_graph.node_link_data(G)
    # 2. Replace each geometry attribute with a GeoJSON dict
    for node in data["nodes"]:
        geom = node.get("geometry")
        if geom is not None:
            # to_geojson returns a JSON string; parse it
            geojson_str = to_geojson(geom, indent=None)
            node["geometry"] = json.loads(geojson_str)
    # 3. Dump to file
    with open(path, "w") as f:
        json.dump(data, f, indent=2)


def graph_from_geojson(path, directed=False, multigraph=True):
    """
    Read a JSON file at 'path' produced by graph_to_geojson and
    reconstruct the original NetworkX graph with Shapely geometries.
    """
    # 1. Load the raw node-link dict
    with open(path) as f:
        data = json.load(f)

    # 2. Convert each node's GeoJSON dict back into a Shapely geometry
    for node_obj in data.get("nodes", []):
        geom_dict = node_obj.get("geometry")
        if isinstance(geom_dict, dict) and "type" in geom_dict:
            node_obj["geometry"] = shape(geom_dict)  # inverse of to_geojson :contentReference[oaicite:1]{index=1}

    # 3. (Optional) If you stored edge geometries similarly, undo those too
    for edge_obj in data.get("links", data.get("edges", [])):
        geom_dict = edge_obj.get("geometry")
        if isinstance(geom_dict, dict) and "type" in geom_dict:
            edge_obj["geometry"] = shape(geom_dict)

    # 4. Rebuild the NetworkX graph (with all attrs, including restored geometries)
    G = json_graph.node_link_graph(
        data,
        directed=directed,
        multigraph=multigraph
    )  # rebuilds Graph from node-link format :contentReference[oaicite:2]{index=2}

    return G


In [8]:
G = graph_from_geojson(path="graphs/lva-railroad-network.json")
m = folium.Map(location=[57.0, 25.0], zoom_start=7)

for node1, node2 in G.edges():
    if node1 in G.nodes and node2 in G.nodes and 'geometry' in G.nodes[node1] and 'geometry' in G.nodes[node2]:
        geom1 = G.nodes[node1]['geometry']
        geom2 = G.nodes[node2]['geometry']
        if isinstance(geom1, Point) and isinstance(geom2, Point):
            lat1, lon1 = geom1.y, geom1.x
            lat2, lon2 = geom2.y, geom2.x
            folium.PolyLine([[lat1, lon1], [lat2, lon2]], color='blue', weight=1).add_to(m)
        else:
            print(f"Warning: Skipping edge between {node1} and {node2} due to non-Point geometry.")
    else:
        print(f"Warning: Skipping edge between {node1} and {node2} because one or both nodes are missing geometry information.")

# Mark the nodes (stations) with red dots
for node, data in G.nodes(data=True):
    if 'geometry' in data and isinstance(data['geometry'], Point):
        folium.CircleMarker(
            location=[data['geometry'].y, data['geometry'].x],
            radius=1,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7
        ).add_to(m)

# Save or display the map
m.save(f'vizualizations/lva_railroads.html')

In [23]:
country_codes: list[str] = [#"aut",
                            # "cze", 
                             "bel", "blr", "nld", "lva"] # "che", "svk", 
geographical_centers: list[list[float]] = [
    #[47.3333, 13.3333],   # Austria (aut)
    #[49.75, 15.5],        # Czechia (cze)
    [50.64111, 4.66806],  # Belgium (bel)
    [53.52904, 28.04497], # Belarus (blr)
    [52.3125, 5.5486],    # Netherlands (nld)
    [57.0, 25.0]          # Latvia (lva)
]
for idx, country in enumerate(country_codes):
    print(country)
    print(geographical_centers[idx])

bel
[50.64111, 4.66806]
blr
[53.52904, 28.04497]
nld
[52.3125, 5.5486]
lva
[57.0, 25.0]


In [ ]:
country_codes: list[str] = [#"aut",
                            # "cze", "che", "svk",
                             "bel", "blr", "nld", "lva"]  
geographical_centers: list[list[float]] = [
    #[47.3333, 13.3333],   # Austria (aut)
    #[49.75, 15.5],        # Czechia (cze)
    [50.64111, 4.66806],  # Belgium (bel)
    [53.52904, 28.04497], # Belarus (blr)
    [52.3125, 5.5486],    # Netherlands (nld)
    [57.0, 25.0]          # Latvia (lva)
]
for idx, country in enumerate(country_codes):
    # loading data
    stations_gdf =  gpd.read_file(f"data/hotosm_{country}_railways_points_geojson.geojson")
    stations_gdf = stations_gdf[stations_gdf['railway'] == 'station']
    tracks_gdf = gpd.read_file(f"data/hotosm_{country}_railways_lines_geojson.geojson")
    tracks_gdf = tracks_gdf[tracks_gdf['railway'] == 'rail'] 
    # graph creation
    G = nx.Graph()

    for index, row in stations_gdf.iterrows():
        if row["name"] and row["geometry"]:
            G.add_node(row["name"], geometry=row["geometry"], id=row["osm_id"])
        
    # Add edges based on tracks
    for idx, track in tracks_gdf.iterrows():
        start_point = Point(track.geometry.coords[0])
        end_point = Point(track.geometry.coords[-1])

        # Find nearest station to start
        start_station = stations_gdf.iloc[stations_gdf.geometry.distance(start_point).idxmin()]
        start_city = start_station['name']

        # Find nearest station to end
        end_station = stations_gdf.iloc[stations_gdf.geometry.distance(end_point).idxmin()]
        end_city = end_station['name']

        # Add edge if cities are different
        if start_city != end_city and start_city and end_city:  # Ensure cities exist
            G.add_edge(start_city, end_city)
    

    graph_to_geojson(G=G, path=f"graphs/{country}-railroad-network.json")
    try:
        m = folium.Map(location=geographical_centers[idx], zoom_start=7)

        for node1, node2 in G.edges():
            if node1 in G.nodes and node2 in G.nodes and 'geometry' in G.nodes[node1] and 'geometry' in G.nodes[node2]:
                geom1 = G.nodes[node1]['geometry']
                geom2 = G.nodes[node2]['geometry']
                if isinstance(geom1, Point) and isinstance(geom2, Point):
                    lat1, lon1 = geom1.y, geom1.x
                    lat2, lon2 = geom2.y, geom2.x
                    folium.PolyLine([[lat1, lon1], [lat2, lon2]], color='blue', weight=1).add_to(m)
                else:
                    print(f"Warning: Skipping edge between {node1} and {node2} due to non-Point geometry.")
            else:
                print(f"Warning: Skipping edge between {node1} and {node2} because one or both nodes are missing geometry information.")

        # Mark the nodes (stations) with red dots
        for node, data in G.nodes(data=True):
            if 'geometry' in data and isinstance(data['geometry'], Point):
                folium.CircleMarker(
                    location=[data['geometry'].y, data['geometry'].x],
                    radius=1,
                    color='red',
                    fill=True,
                    fill_color='red',
                    fill_opacity=0.7
                ).add_to(m)

        # Save or display the map
        m.save(f'vizualizations/{country}_railroads.html')
    except:
        pass

C:\Users\pazera\AppData\Local\Temp\ipykernel_22832\906148296.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  start_station = stations_gdf.iloc[stations_gdf.geometry.distance(start_point).idxmin()]
C:\Users\pazera\AppData\Local\Temp\ipykernel_22832\906148296.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  end_station = stations_gdf.iloc[stations_gdf.geometry.distance(end_point).idxmin()]
C:\Users\pazera\AppData\Local\Temp\ipykernel_22832\906148296.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  start_station = stations_gdf.iloc[stations_gdf.geometry.distanc

In [ ]:
geographical_centers[0]

In [ ]:
import networkx as nx
from shapely.geometry import Point

G = nx.Graph()

for index, row in stations_gdf.iterrows():
    G.add_node(row["name"], geometry=row["geometry"], id=row["osm_id"])
    
# Add edges based on tracks
for idx, track in tracks_gdf.iterrows():
    start_point = Point(track.geometry.coords[0])
    end_point = Point(track.geometry.coords[-1])

    # Find nearest station to start
    start_station = stations_gdf.iloc[stations_gdf.geometry.distance(start_point).idxmin()]
    start_city = start_station['name']

    # Find nearest station to end
    end_station = stations_gdf.iloc[stations_gdf.geometry.distance(end_point).idxmin()]
    end_city = end_station['name']

    # Add edge if cities are different
    if start_city != end_city and start_city and end_city:  # Ensure cities exist
        G.add_edge(start_city, end_city)

In [ ]:
m = folium.Map(location=[48.7, 19.5], zoom_start=7)

for node1, node2 in G.edges():
    if node1 in G.nodes and node2 in G.nodes and 'geometry' in G.nodes[node1] and 'geometry' in G.nodes[node2]:
        geom1 = G.nodes[node1]['geometry']
        geom2 = G.nodes[node2]['geometry']
        if isinstance(geom1, Point) and isinstance(geom2, Point):
            lat1, lon1 = geom1.y, geom1.x
            lat2, lon2 = geom2.y, geom2.x
            folium.PolyLine([[lat1, lon1], [lat2, lon2]], color='blue', weight=1).add_to(m)
        else:
            print(f"Warning: Skipping edge between {node1} and {node2} due to non-Point geometry.")
    else:
        print(f"Warning: Skipping edge between {node1} and {node2} because one or both nodes are missing geometry information.")

# Mark the nodes (stations) with red dots
for node, data in G.nodes(data=True):
    if 'geometry' in data and isinstance(data['geometry'], Point):
        folium.CircleMarker(
            location=[data['geometry'].y, data['geometry'].x],
            radius=1,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7
        ).add_to(m)

# Save or display the map
m.save('slovakia_railroads.html')

In [ ]:
import json
from shapely import to_geojson
from shapely.geometry import shape
from networkx.readwrite import json_graph

def graph_to_geojson(G, path):
    """
    Serialize NetworkX graph G (with Shapely geometries in node attrs)
    to a JSON file at 'path', converting geometries via to_geojson.
    """
    # 1. Extract node-link data
    data = json_graph.node_link_data(G)
    # 2. Replace each geometry attribute with a GeoJSON dict
    for node in data["nodes"]:
        geom = node.get("geometry")
        if geom is not None:
            # to_geojson returns a JSON string; parse it
            geojson_str = to_geojson(geom, indent=None)
            node["geometry"] = json.loads(geojson_str)
    # 3. Dump to file
    with open(path, "w") as f:
        json.dump(data, f, indent=2)


def graph_from_geojson(path, directed=False, multigraph=True):
    """
    Read a JSON file at 'path' produced by graph_to_geojson and
    reconstruct the original NetworkX graph with Shapely geometries.
    """
    # 1. Load the raw node-link dict
    with open(path) as f:
        data = json.load(f)

    # 2. Convert each node's GeoJSON dict back into a Shapely geometry
    for node_obj in data.get("nodes", []):
        geom_dict = node_obj.get("geometry")
        if isinstance(geom_dict, dict) and "type" in geom_dict:
            node_obj["geometry"] = shape(geom_dict)  # inverse of to_geojson :contentReference[oaicite:1]{index=1}

    # 3. (Optional) If you stored edge geometries similarly, undo those too
    for edge_obj in data.get("links", data.get("edges", [])):
        geom_dict = edge_obj.get("geometry")
        if isinstance(geom_dict, dict) and "type" in geom_dict:
            edge_obj["geometry"] = shape(geom_dict)

    # 4. Rebuild the NetworkX graph (with all attrs, including restored geometries)
    G = json_graph.node_link_graph(
        data,
        directed=directed,
        multigraph=multigraph
    )  # rebuilds Graph from node-link format :contentReference[oaicite:2]{index=2}

    return G


In [ ]:
graph_to_geojson(G=G, path="Slovak-railroad-network.json")

In [ ]:
G_re_read = graph_from_geojson("Slovak-railroad-network.json")
del G_re_read

In [ ]:
stations_gdf_che = gpd.read_file('hotosm_che_railways_points_geojson.geojson')
stations_gdf_che = stations_gdf_che[stations_gdf_che['railway'] == 'station']  # Filter for stations

tracks_gdf_che = gpd.read_file('hotosm_che_railways_lines_geojson.geojson')
tracks_gdf_che = tracks_gdf_che[tracks_gdf_che['railway'] == 'rail']  # Filter for railway tracks

In [ ]:
G_che= nx.Graph()

for index, row in stations_gdf_che.iterrows():
    G_che.add_node(row["name"], geometry=row["geometry"], id=row["osm_id"])
    
# Add edges based on tracks
for idx, track in tracks_gdf_che.iterrows():
    start_point = Point(track.geometry.coords[0])
    end_point = Point(track.geometry.coords[-1])

    # Find nearest station to start
    start_station = stations_gdf_che.iloc[stations_gdf_che.geometry.distance(start_point).idxmin()]
    start_city = start_station['name']

    # Find nearest station to end
    end_station = stations_gdf_che.iloc[stations_gdf_che.geometry.distance(end_point).idxmin()]
    end_city = end_station['name']

    # Add edge if cities are different
    if start_city != end_city and start_city and end_city:  # Ensure cities exist
        G_che.add_edge(start_city, end_city)


In [ ]:
nodes_to_remove = []
for index, row in stations_gdf.iterrows():
    nodes_to_remove.append(row["name"])

G_che.remove_nodes_from(nodes_to_remove)

graph_to_geojson(G=G_che, path="Switzerland-railroad-network.json")

In [ ]:
# switzerland first vizualization
m = folium.Map(location=[46.81, 8.20], zoom_start=7)

for node1, node2 in G_che.edges():
    if node1 in G_che.nodes and node2 in G_che.nodes and 'geometry' in G_che.nodes[node1] and 'geometry' in G_che.nodes[node2]:
        geom1 = G_che.nodes[node1]['geometry']
        geom2 = G_che.nodes[node2]['geometry']
        if isinstance(geom1, Point) and isinstance(geom2, Point):
            lat1, lon1 = geom1.y, geom1.x
            lat2, lon2 = geom2.y, geom2.x
            folium.PolyLine([[lat1, lon1], [lat2, lon2]], color='blue', weight=1).add_to(m)
        else:
            print(f"Warning: Skipping edge between {node1} and {node2} due to non-Point geometry.")
    else:
        print(f"Warning: Skipping edge between {node1} and {node2} because one or both nodes are missing geometry information.")

# Mark the nodes (stations) with red dots
for node, data in G_che.nodes(data=True):
    if 'geometry' in data and isinstance(data['geometry'], Point):
        folium.CircleMarker(
            location=[data['geometry'].y, data['geometry'].x],
            radius=1,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7
        ).add_to(m)

# Save or display the map
m.save('Switzerland_railroads.html')